In [1]:
import ipywidgets
from IPython.display import display
from jetcam.utils import bgr8_to_jpeg
from jetcam.csi_camera import CSICamera

import argparse
import json
import cv2
import numpy as np

from yolo.frontend import create_yolo
from yolo.backend.utils.box import draw_scaled_boxes
from yolo.backend.utils.annotation import parse_annotation
from yolo.backend.utils.eval.fscore import count_true_positives, calc_score

from pascal_voc_writer import Writer
from shutil import copyfile
import os
import yolo
import time


DEFAULT_CONFIG_FILE = os.path.join(yolo.PROJECT_ROOT, "svhn", "config.json")
DEFAULT_WEIGHT_FILE = os.path.join(yolo.PROJECT_ROOT, "svhn", "weights.h5")
DEFAULT_THRESHOLD = 0.3


/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/dtypes.py:519: FutureWarning: Passing (type, 1) or 

In [2]:


with open('turn4/config.json') as config_buffer:
    config = json.loads(config_buffer.read())

# 2. create yolo instance & predict
yolo = create_yolo(config['model']['architecture'],
                   config['model']['labels'],
                   config['model']['input_size'],
                   config['model']['anchors'])
yolo.load_weights('model.h5')



Model: "model_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 224, 224, 3)       0         
_________________________________________________________________
mobilenet_0.75_224 (Model)   (None, 7, 7, 768)         1832976   
_________________________________________________________________
detection_layer_50 (Conv2D)  (None, 7, 7, 50)          38450     
_________________________________________________________________
reshape_1 (Reshape)          (None, 7, 7, 5, 10)       0         
Total params: 1,871,426
Trainable params: 1,855,010
Non-trainable params: 16,416
_________________________________________________________________
Loading pre-trained weights in model.h5


In [3]:

from IPython.display import clear_output

In [4]:
camera = CSICamera(width=320, height=240, capture_fps=120)
camera.running = 0
camera.running = True

TraitError: The 'running' trait of a CSICamera instance expected a boolean, not the int 0.

In [7]:
image = camera.value
image_widget = ipywidgets.Image(format='jpeg')
label_widget = ipywidgets.Label()
image_widget.value = bgr8_to_jpeg(image)

display(image_widget, label_widget)

Image(value=b'\xff\xd8\xff\xe0\x00\x10JFIF\x00\x01\x01\x00\x00\x01\x00\x01\x00\x00\xff\xdb\x00C\x00\x02\x01\x0…

Label(value='hello')

In [ ]:
timer = 0
timer2 = 0
while 1 :
    timer2 = time.time()
    image  = camera.value
    image = image[8:8+224, 48:48+224]
    boxes, probs = yolo.predict(image, float(0.6))
    # labels = np.argmax(probs, axis=1)
    # if len(probs) > 0 else [] 
    timer = time.time() - timer2

    image = draw_scaled_boxes(image, boxes, probs, config['model']['labels'])
    image_widget.value = bgr8_to_jpeg(image)
    label_widget.value = "frame rates: " + str(int(1/timer)) + " fps"
#     clear_output()
#     print(timer)
    


In [ ]:
display(label)